In [9]:
import numpy as np
import pandas as pd
import random
from random import randint
import string
from pyspark.sql.types import *
from pyspark.sql.functions import when
from pyspark.ml.feature import Imputer
from datetime import datetime, timedelta

In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("sourcedata").getOrCreate()

In [11]:
rec_cnt=1001
dataset_date="2022-01-03"
data_dir="/home/cloudera/projects/notebook/data/"
file_name=data_dir+"cards_account_ingest_"+dataset_date+".csv"
columns=['account_id','account_open_dt','account_id_type','acct_hldr_primary_addr_state',\
            'acct_hldr_primary_addr_zip_cd','acct_hldr_first_name','acct_hldr_last_name','dataset_date']

In [12]:
def getRandomDates():
    start = datetime.now()
    end = start + timedelta(days=700)
    random_date = (start - (end - start) * random.random()).strftime('%Y-%m-%d')
    return random_date

In [13]:
def get_random_string(length):
    # choose from all lowercase letter
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    return result_str

In [14]:
def build_schema():
    print("Build and return a schema to use for the sample data.")
    schema = StructType(
        [
            StructField("account_id",  StringType(),  True),
            StructField("account_open_dt",      StringType(),  True),
            StructField("account_id_type", StringType(),  True),
            StructField("acct_hldr_primary_addr_state", StringType(),  True),
            StructField("acct_hldr_primary_addr_zip_cd",    StringType(),  True),
            StructField("acct_hldr_first_name", StringType(),  True),
            StructField("acct_hldr_last_name", StringType(),  True),
            StructField("dataset_date",  StringType(),  True)
        ]
    )
    return schema

In [15]:

def build_data(cnt):
    rec = []
    for i in range(1,rec_cnt):
        account_id =  random.choice(["SV","CK","PV"])+str(randint(27618,189765))
        account_open_dt = getRandomDates()
        acct_hldr_primary_addr_zip_cd=str(randint(23456,98776))
        acct_hldr_primary_addr_state = random.choice(['AL','AK','AZ','AR','CA','CO','CT','DE','DC',\
                                                      'FL','GA','HI','ID','IL','IN','IA','KS','KY','LA',\
                                                      'ME','MD','MA'])
        account_id_type = random.choice(["Saving","Checkin","Private"])
        acct_hldr_first_name=get_random_string(10)
        acct_hldr_last_name=get_random_string(6)
        rec.append(account_id + "~" +
                   account_open_dt + "~" +
                   account_id_type + "~" +
                   acct_hldr_primary_addr_state + "~" +
                   acct_hldr_primary_addr_zip_cd + "~" +
                   acct_hldr_first_name + "~" +
                   acct_hldr_last_name + "~" +
                   dataset_date)
    return (rec)


hist_data = build_data(int(rec_cnt))
histrawInput = sc.parallelize(hist_data)
histrawInputSplit = histrawInput.map(lambda x: x.split("~"))
hist_df = spark.createDataFrame(histrawInputSplit, schema=build_schema())
hist_df.toPandas().to_csv(file_name,index=False)

Build and return a schema to use for the sample data.


In [16]:
# hist_df.toPandas().to_csv(file_name,index=False)